Following is needed for connecting to google drive and setting up fairseq to for tokenization and binarization.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install tensorboardX
! pip install tensorrt

In [ ]:
!pip install cython -U
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --quiet --editable .
!pip install --quiet sentencepiece

In [ ]:
! mkdir -p gpt2_bpe
! wget -O gpt2_bpe/encoder.json https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
! wget -O gpt2_bpe/vocab.bpe https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
! wget -O gpt2_bpe/dict.txt https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt

# Truncation

**Run only once!!!** (r2 added for failsafe)

In [ ]:
import numpy as np
import random
f1 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/train.raw.input0', 'r')
f2 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/train.raw.input1', 'r')
f3 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/train.raw.label', 'r')

o1 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/trainr2.raw.input0', 'w')
o2 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/trainr2.raw.input1', 'w')
o3 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/trainr2.raw.label', 'w')


lines1 = f1.read().splitlines()
lines2 = f2.read().splitlines()
lines3 = f3.read().splitlines()

arr = random.sample(range(1, len(lines1)), 10000)

for i in arr:
    o1.write(lines1[i].rstrip()+"\n")
    o2.write(lines2[i].rstrip()+"\n")
    o3.write(lines3[i].rstrip()+"\n")

o1.close()
o2.close()
o3.close()

# No augmentation

Target: \bin


In [ ]:
!for GENRE in fiction; do \
    for SPLIT in trainr valid; do \
      for TYPE in input0 input1; do \
        python /content/fairseq/examples/roberta/multiprocessing_bpe_encoder.py \
            --encoder-json gpt2_bpe/encoder.json \
            --vocab-bpe gpt2_bpe/vocab.bpe \
            --inputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/${SPLIT}.raw.${TYPE} \
            --outputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/${SPLIT}.bpe.${TYPE} \
            --keep-empty \
            --workers 60; \
      done \
    done \
  done


In [ ]:
!for GENRE in fiction; do \
    for TYPE in input0 input1; do \
      fairseq-preprocess \
      --only-source \
      --srcdict gpt2_bpe/dict.txt \
      --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/trainr.bpe.${TYPE} \
      --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.bpe.${TYPE} \
      --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.bpe.${TYPE} \
      --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin/${TYPE} \
      --workers 60;\
    done \
  done

In [ ]:
!for GENRE in fiction; do \
    for TYPE in label; do \
      fairseq-preprocess \
      --only-source \
      --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/trainr.raw.${TYPE} \
      --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.raw.${TYPE} \
      --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.raw.${TYPE} \
      --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin/${TYPE} \
      --workers 60;\
    done \
  done

# EDA

Target: \bin_eda

In [ ]:
import nltk; nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import subprocess

def execute_shell_command(command):
    process = subprocess.Popen(command, shell=True)
    process.wait()

command = """
for GENRE in fiction; do \
    for TYPE in input0 input1; do \
        paste -d "\t" /content/drive/MyDrive/NLP/mnli/$GENRE/orig/trainr.raw.label /content/drive/MyDrive/NLP/mnli/$GENRE/orig/trainr.raw.$TYPE > /content/drive/MyDrive/NLP/mnli/$GENRE/orig/eda/train.combined.label.$TYPE; \
    done \
done
"""

execute_shell_command(command)

In [ ]:
import subprocess

def execute_shell_command(command):
    process = subprocess.Popen(command, shell=True)
    process.wait()

genres = ['fiction']
types = ['input0', 'input1']

for genre in genres:
    for type_ in types:
        command = f"python /content/drive/MyDrive/NLP/eda_nlp-master/code/augment.py \
                    --input /content/drive/MyDrive/NLP/mnli/{genre}/orig/eda/train.combined.label.{type_} \
                    --num_aug=4"
        execute_shell_command(command)

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  !for GENRE in fiction; do \
    for SPLIT in train; do \
      for TYPE in input0 input1; do \
        python /content/fairseq/examples/roberta/multiprocessing_bpe_encoder.py \
            --encoder-json gpt2_bpe/encoder.json \
            --vocab-bpe gpt2_bpe/vocab.bpe \
            --inputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda/eda_${SPLIT}.combined.label.${TYPE} \
            --outputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda/eda_${SPLIT}.combined.label.bpe.${TYPE} \
            --keep-empty \
            --workers 60; \
      done \
    done \
  done

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  !for GENRE in fiction; do \
    for SPLIT in valid test; do \
      for TYPE in input0 input1; do \
        python /content/fairseq/examples/roberta/multiprocessing_bpe_encoder.py \
            --encoder-json gpt2_bpe/encoder.json \
            --vocab-bpe gpt2_bpe/vocab.bpe \
            --inputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/${SPLIT}.raw.${TYPE} \
            --outputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/${SPLIT}.bpe.${TYPE} \
            --keep-empty \
            --workers 60; \
      done \
    done \
  done

In [ ]:
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in input0 input1; do \
        fairseq-preprocess \
        --only-source \
        --srcdict gpt2_bpe/dict.txt \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda/eda_train.combined.label.bpe.${TYPE}  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.bpe.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.bpe.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_eda2/${TYPE} \
        --workers 60;\
      done \
    done


In [ ]:
  # for the two input scentences
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in label; do \
        fairseq-preprocess \
        --only-source \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda/eda_label_train.combined.${TYPE}.input0  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.raw.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.raw.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_eda2/${TYPE} \
        --workers 60;\
      done \
    done

# UDA

Target: \bin_uda

In [ ]:
! pip install transformers
! pip install sentencepiece

In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer

torch.cuda.empty_cache()

en_fr_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
en_fr_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

fr_en_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
fr_en_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

In [ ]:
# Code for translating input 0
input = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/trainr2.raw.input0', 'r')
src_text = input.read().splitlines()

output = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/translate/trans_train.raw.input0', 'w')

for line in src_text:
  translated_tokens = en_fr_model.generate(**en_fr_tokenizer(line, return_tensors="pt", padding=True), num_return_sequences = 2, top_k = 10, temperature = 2.0)
  en_fr = [en_fr_tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

  translated_tokens2 = fr_en_model.generate(**fr_en_tokenizer(en_fr, return_tensors="pt", padding=True), num_return_sequences = 2, top_k = 10, temperature = 2.0)
  fr_en = [fr_en_tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens2]
  output.write(line.rstrip()+"\n")

  for r in fr_en:
    output.write(r.rstrip()+"\n")
    output.flush()

input.close()
output.close()

In [ ]:
# Code for translating input 1
input = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/trainr2.raw.input1', 'r')
src_text = input.read().splitlines()

output = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/translate/trans_train.raw.input1', 'w')

for line in src_text:
  translated_tokens = en_fr_model.generate(**en_fr_tokenizer(line, return_tensors="pt", padding=True), num_return_sequences = 2, top_k = 10, temperature = 2.0)
  en_fr = [en_fr_tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

  translated_tokens2 = fr_en_model.generate(**fr_en_tokenizer(en_fr, return_tensors="pt", padding=True), num_return_sequences = 2, top_k = 10, temperature = 2.0)
  fr_en = [fr_en_tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens2]
  output.write(line.rstrip()+"\n")

  for r in fr_en:
    output.write(r.rstrip()+"\n")
    output.flush()

input.close()
output.close()

In [ ]:
# create label file
f = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/trainr.raw.label', 'r')
o = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/translate/trans_train.raw.label', 'w')

lines = f.read().splitlines()
print(len(lines))
for i in lines:
  for j in range (5):
    o.write(i.strip()+"\n")


o.close()

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  !for GENRE in fiction; do \
    for SPLIT in train; do \
      for TYPE in input0 input1; do \
        python /content/fairseq/examples/roberta/multiprocessing_bpe_encoder.py \
            --encoder-json gpt2_bpe/encoder.json \
            --vocab-bpe gpt2_bpe/vocab.bpe \
            --inputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/translate/trans_${SPLIT}.raw.${TYPE} \
            --outputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/translate/trans_${SPLIT}.raw.bpe.${TYPE} \
            --keep-empty \
            --workers 60; \
      done \
    done \
  done

In [ ]:
  # for the two input scentences
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in input0 input1; do \
        fairseq-preprocess \
        --only-source \
        --srcdict gpt2_bpe/dict.txt \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/translate/trans_train.raw.bpe.${TYPE}  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.bpe.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.bpe.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_uda/${TYPE} \
        --workers 60;\
      done \
    done

In [ ]:
  # for the labels
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in label; do \
        fairseq-preprocess \
        --only-source \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/translate/trans_train.raw.${TYPE}  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.raw.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.raw.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_uda/${TYPE} \
        --workers 60;\
      done \
    done

# SSMBA

Target: \bin_ssmba

In [ ]:
! git clone https://github.com/nng555/ssmba.git /content/drive/MyDrive/ssmba
! pip install -r /content/drive/MyDrive/ssmba/requirements.txt

In [ ]:
import subprocess
import tensorflow as tf

def execute_shell_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    stdout, stderr = process.communicate()
    return process.returncode, stdout, stderr

with tf.device('/device:GPU:0'):
    for genre in ['fiction']:
        for split in ['trainr']:
            for type_ in ['input0 input1']:
                command = [
                    'python',
                    '/content/drive/MyDrive/ssmba/ssmba.py',
                    '--model', 'bert-base-uncased',
                    '--in-file', f'/content/drive/MyDrive/{genre}/orig/{split}.raw.{type_}',
                    '--label-file', f'/content/drive/MyDrive/{genre}/orig/{split}.raw.label',
                    '--output-prefix', f'ssmba_out',
                    '--noise-prob', '0.05',
                    '--num-samples', '5',
                ]
                returncode, stdout, stderr = execute_shell_command(command)
                if returncode != 0:
                    print(f"Error running the command for {genre}/{split}/{type_}:")
                    print(stderr)
                else:
                    print(f"Command executed successfully for {genre}/{split}/{type_}:")
                    print(stdout)

In [ ]:
f = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/ssmba/ssmba_out', 'r')
o1 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/ssmba/ssmba_train.raw.input0', 'w')
o2 = open('/content/drive/MyDrive/NLP/mnli/fiction/orig/ssmba/ssmba_train.raw.input1', 'w')

lines = [tuple(s.strip().split('\t')) for s in f.readlines()]

for i in range (len(lines)):
  o1.write(lines[i][0].strip()+"\n")
  o2.write(lines[i][1].strip()+"\n")


o1.close()
o2.close()

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  !for GENRE in fiction; do \
    for SPLIT in train; do \
      for TYPE in input0 input1; do \
        python /content/fairseq/examples/roberta/multiprocessing_bpe_encoder.py \
            --encoder-json gpt2_bpe/encoder.json \
            --vocab-bpe gpt2_bpe/vocab.bpe \
            --inputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/ssmba/ssmba_${SPLIT}.raw.${TYPE} \
            --outputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/ssmba/ssmba_${SPLIT}.raw.bpe.${TYPE} \
            --keep-empty \
            --workers 60; \
      done \
    done \
  done

In [ ]:
  # for the two input scentences
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in input0 input1; do \
        fairseq-preprocess \
        --only-source \
        --srcdict gpt2_bpe/dict.txt \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/ssmba/ssmba_train.raw.bpe.${TYPE}  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.bpe.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.bpe.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_ssmba2/${TYPE} \
        --workers 60;\
      done \
    done

In [ ]:
  # for the labels
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in label; do \
        fairseq-preprocess \
        --only-source \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/ssmba/ssmba_out2.${TYPE}  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.raw.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.raw.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_ssmba2/${TYPE} \
        --workers 60;\
      done \
    done

# EDA_Extended

In [ ]:
import nltk; nltk.download('wordnet')

In [ ]:
! pip install python-utils

In [ ]:
# First Experiment
import subprocess

def execute_shell_command(command):
    process = subprocess.Popen(command, shell=True)
    process.wait()

command = """
for GENRE in fiction; do \
    for TYPE in input0 input1; do \
        paste -d "\t" /content/drive/MyDrive/NLP/mnli/$GENRE/orig/trainr.raw.label /content/drive/MyDrive/NLP/mnli/$GENRE/orig/trainr.raw.$TYPE > /content/drive/MyDrive/NLP/mnli/$GENRE/orig/eda_extended/train.combined.label.$TYPE; \
    done \
done
"""

execute_shell_command(command)

In [ ]:
# First Experiment
!for GENRE in fiction; do \
  for TYPE in input0 input1; do \
    python /content/drive/MyDrive/NLP/eda_extended/augment.py \
      --input /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda_extended/train.combined.label.${TYPE} \
      --num_aug=4 \
      --alpha_sr=0.1 \
      --alpha_ri=0.1 \
      --alpha_rs=0.1 \
      --alpha_rd=0.1; \
    done \
  done

In [ ]:
# First Experiment
import tensorflow as tf
with tf.device('/device:GPU:0'):
  !for GENRE in fiction; do \
    for SPLIT in train; do \
      for TYPE in input0 input1; do \
        python /content/fairseq/examples/roberta/multiprocessing_bpe_encoder.py \
            --encoder-json gpt2_bpe/encoder.json \
            --vocab-bpe gpt2_bpe/vocab.bpe \
            --inputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda_extended/eda_extended_${SPLIT}.combined.label.${TYPE} \
            --outputs /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda_extended/eda_extended_${SPLIT}.combined.label.bpe.${TYPE} \
            --keep-empty \
            --workers 60; \
      done \
    done \
  done

In [ ]:
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in input0 input1; do \
        fairseq-preprocess \
        --only-source \
        --srcdict gpt2_bpe/dict.txt \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda_extended/eda_extended_train.combined.label.bpe.${TYPE}  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.bpe.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.bpe.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_eda_extended/${TYPE} \
        --workers 60;\
      done \
    done


In [ ]:
  # for the two input scentences
with tf.device('/device:GPU:0'):
    !for GENRE in fiction; do \
      for TYPE in label; do \
        fairseq-preprocess \
        --only-source \
        --trainpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/eda_extended/eda_label_extended_train.combined.${TYPE}.input0  \
        --validpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/valid.raw.${TYPE} \
        --testpref /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/test.raw.${TYPE} \
        --destdir /content/drive/MyDrive/NLP/mnli/${GENRE}/orig/bin_eda_extended/${TYPE} \
        --workers 60;\
      done \
    done